# Trendlines Validation Method
Objective:
Find a reliable method to validate the calculated trend curves.
Current method in production:
- Validation based on a minimum of 20 different vehicles.

Current approach being explored:
- Use at least 50 different vehicles, including:
    - 20 vehicles with more than 100,000 km
    - 20 vehicles with less than 80,000 km


In [ ]:
from core.gsheet_utils import *
from core.sql_utils import *
from core.sql_utils import get_connection, get_sqlalchemy_engine
import pandas as pd
from core.gsheet_utils import load_excel_data
from sqlalchemy import text
from load.trendline.trendline_utils import filtrer_trendlines
from load.trendline.main import generate_trendline_functions
from core.plt_utils import show_trendline
import re
from core.sql_utils import *
from activation.config.mappings import mapping_vehicle_type

### Load data

In [ ]:

engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT vm.model_name, vm.id, vm.type, m.make_name, b.capacity FROM vehicle_model vm
                                  join make m on m.id=vm.make_id
                                  join battery b on b.id=vm.battery_id;"""), con)


In [ ]:

with engine.connect() as connection:
    prod_df = pd.read_sql(text("""SELECT v.vin, vm.model_name, vm.type, vd.odometer, vd.soh, vm.version  from vehicle_data vd
join vehicle v
on vd.vehicle_id=v.id
join vehicle_model vm
on v.vehicle_model_id=vm.id;"""), con)


In [ ]:
df = load_excel_data("Courbes de tendance", "Courbes OS")
df_sheet = pd.DataFrame(columns=df[0,:8], data=df[1:,:8])
df_sheet['SoH'] = df_sheet['SoH'].apply(lambda x:  x.replace('%', '').strip()).astype(float) / 100

In [ ]:
df_sheet['model_id'] = df_sheet.apply(lambda row: mapping_vehicle_type(row['Type'], row['OEM'], 
                                                                       row['Modèle'], dbeaver_df, row['battery_capacity']), axis=1)
df_sheet['model_id'] = df_sheet['model_id'].astype(str)

In [ ]:
df_sheet['Odomètre (km)'] = df_sheet['Odomètre (km)'].apply(lambda x: float(x.replace(',', '')))

## Check conditioning

In [ ]:
resultats = []
for modele, group in df_sheet.groupby(["Modèle", 'model_id']):
    nb_total_vins = group['lien'].nunique()
    nb_moins_50k = group[group['Odomètre (km)'] <= 80000]['lien'].nunique()
    nb_plus_80k = group[group['Odomètre (km)'] >= 100000]['lien'].nunique()
    
    if nb_total_vins >= 50 and nb_moins_50k >= 20 and nb_plus_80k >= 10:
        resultats.append(modele)
for i in resultats:
    print(i)

In [ ]:
resultats = []
for modele, group in prod_df.groupby(["model_name", 'type', 'version']):
    nb_total_vins = group['vin'].nunique()
    nb_moins_50k = group[group['odometer'] <= 80_000]['vin'].nunique()
    nb_plus_80k = group[group['odometer'] >= 100_000]['vin'].nunique()
    
    if nb_total_vins >= 50 and nb_moins_50k >= 20 and nb_plus_80k >= 20:
        resultats.append(modele)

for i in resultats:
    print(i)

Conlusion:

If we lower the threshold to 20 VINs under 50,000 km and 20 over 50,000 km, we can include two model types from scrapping (R110 and e-208 50kWh 1st gen) as well as Tesla and Zoe R135 from the bib SoH.

However, if we maintain the current approach, we only have Tesla data from the bib SoH.



## Oem trendline

In [ ]:
with get_connection() as connection:
    query = """
        SELECT vm.model_name, vm.id as vehicle_model_id, vm.type, m.make_name, vd.soh, vd.soh_oem, vd.odometer, o.oem_name, o.id as oem_id, m.id as make_id FROM vehicle v
        left JOIN vehicle_model vm ON vm.id = v.vehicle_model_id
        left JOIN vehicle_data vd ON vd.vehicle_id = v.id
        left JOIN oem o ON o.id = vm.oem_id
        left JOIN battery b ON b.id = vm.battery_id
        left JOIN make m on m.id = vm.make_id
        where (soh is not NULL or soh_oem is not null)
        and o.id not in (uuid('fe6fba2f-1e10-43a7-b8b0-2aa021f97c3f'), uuid('1b093647-0d1b-409f-8b37-871bc81ac9ca'), uuid('5d2e8c26-d2bb-4c24-812e-59c3a7508213'), uuid('fc285a82-f5ea-44a0-8f2e-706cd5f8c868'))
    """
    df = pd.read_sql(query, connection)
    df['soh_oem'] = pd.to_numeric(df['soh_oem'], errors='coerce')   
    df['soh'] = pd.to_numeric(df['soh'], errors='coerce')
    df['soh'] = df.apply(lambda x: x['soh_oem'] / 100 if float(x['soh_oem']) > 0 else x['soh'], axis=1)
    df = df[df['soh'] < 1.1]
    df['origin'] = "dbeaver"

    

df_sheet = load_excel_data("Courbes de tendance", "Courbes OS")
df_sheet = pd.DataFrame(data=df_sheet[1:,:8], columns=["make_name", "model_name", "type", "year", "odometer", "soh", 'link', "battery_capacity"])
df_sheet['model_name'] = df_sheet['model_name'].apply(str.lower)
df_sheet['make_name'] = df_sheet['make_name'].apply(str.lower)
df_sheet["soh"] = df_sheet["soh"].apply(lambda x:  x.replace('%', '').strip()).astype(float) / 100
df_sheet["odometer"] = df_sheet["odometer"].apply(lambda x:  str(x).replace(',', '').strip()).astype(float)
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT m.make_name, o.id as oem_id, o.oem_name, m.id as make_id FROM make m
                                join oem o on o.id=m.oem_id;"""), con)
    df_merge = df_sheet.merge(dbeaver_df, on=['make_name',], how='left')
    df_merge['origin'] = "scrapping"

df_all = pd.concat((df, df_merge), ignore_index=True)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
def trendline_apply(x, f):
    return eval(f)

def show_trendline(df, trendline, trendline_max, trendline_min, model, odometer_column_name, soh_column_name):
    palette = px.colors.qualitative.Plotly
    unique = df['origin'].astype(str).fillna('Unknown').unique()
    color_map = {u: palette[i % len(palette)] for i, u in enumerate(unique)}
    marker_colors = df['origin'].astype(str).fillna('Unknown').map(color_map)
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df[odometer_column_name],
        y=df[soh_column_name],
        mode='markers',
        marker=dict(color=marker_colors, size=7),
        name='SoH'
    ))

    x_sorted = df[odometer_column_name].sort_values()
    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline['trendline']),
        mode='lines',
        line=dict(color='red'),
        name='Fit'
    ))

    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline_max['trendline']),
        mode='lines',
        line=dict(color='green'),
        name='Upper'
    ))

    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline_min['trendline']),
        mode='lines',
        line=dict(color='green'),
        name='Lower'
    ))

    fig.update_layout(
        width=1000,
        height=600,
        xaxis_title='Odometer',
        yaxis_title='State of Health (SoH)',
        legend_title='Légende',
        title=f"version: {model}",
        template='plotly_white',
        xaxis=dict(range=[0, 150000]),  # Change selon l'échelle souhaitée pour l'odomètre
        yaxis=dict(range=[.75, 1.1])     # Change selon l'échelle souhaitée pour le SoH
    )

    return fig

In [ ]:
for oem in df_all.oem_id.unique():
    df_temp = df_all[df_all['oem_id']==oem]
    if filtrer_trendlines(df_temp,  "odometer", "odometer", 50_000, 50_000, 50, 10, 10):
        mean_trend, upper_bound, lower_bound = generate_trendline_functions(df_temp, "odometer", "soh")
        fig = show_trendline(df_temp, mean_trend, upper_bound, lower_bound, df_temp.oem_name.unique(), 'odometer', 'soh')
        fig.show()
    else:
        print(df_temp.oem_id.unique())